In [ ]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import Draw
import networkx as nx
from torch_geometric.datasets import MoleculeNet
import matplotlib.pyplot as plt
import torch
import os
import pandas as pd
import numpy as np
import json,pickle
from collections import OrderedDict
from rdkit import Chem
from rdkit.Chem import MolFromSmiles
import networkx as nx

from torch_geometric.data import InMemoryDataset, Data
from torch_geometric.loader import DataLoader

from sklearn.model_selection import train_test_split
from tqdm.notebook import tqdm
import torch
import torch.nn as nn

import Data_Preprocessing.Graph_Data as gd
from Data_Preprocessing.Graph_Data import Molecule_data
from models.AttentiveFPModel import AttentiveFP

In [ ]:
def createFoldsData():
    iy = 0
    folds = 5
    for fold in tqdm(range(folds)):
        df_train = pd.read_csv('New_fold/fold_'+str(iy)+'_'+'x_train.csv')
        df_test  = pd.read_csv('New_fold/fold_'+str(iy)+'_'+'x_test.csv')
        smiles = df_train['SMILE']
        codIds = df_train['CODID']
        band_gap = df_train['bgs']
        band_gap = band_gap.to_numpy()

        smiles_test = df_test['SMILE']
        codIds_test = df_test['CODID']
        band_gap_test = df_test['bgs']
        band_gap_test = band_gap_test.to_numpy()


        smile_graph = {}
        band_gap_arr = []
        smiles_array = []

        for i,smile in enumerate(smiles):
            g = gd.smile_to_graph(smile)
            if g != None:
                smile_graph[smile] = g
                band_gap_arr.append(band_gap[i])
                smiles_array.append(smile)

        smile_graph_test = {}
        band_gap_arr_test = []
        smiles_array_test = []

        for i,smile in enumerate(smiles_test):
            g = gd.smile_to_graph(smile)
            if g != None:
                smile_graph_test[smile] = g
                band_gap_arr_test.append(band_gap_test[i])
                smiles_array_test.append(smile)

        train_data = Molecule_data(root='data', dataset='train_data_set_fold_'+str(iy),y=band_gap_arr,
                                   smile_graph=smile_graph,smiles=smiles_array)

        test_data = Molecule_data(root='data', dataset='test_data_set_fold_'+str(iy),y=band_gap_arr_test,
                                   smile_graph=smile_graph_test,smiles=smiles_array_test)

        iy+=1

In [ ]:
df = pd.read_csv('Data_Preprocessing/omdb_smile_data_set.csv')

In [ ]:
smiles = df['SMILE']
codIds = df['CODID']
band_gap = df['bgs']

In [ ]:
band_gap = band_gap.to_numpy()

In [ ]:
def createFoldsCsv():
    from sklearn.model_selection import StratifiedKFold
    from sklearn.model_selection import KFold
    kf = KFold(n_splits=5,shuffle = True, random_state = 42)
    ix = 0
    train1 = df
    for train_index, test_index in (kf.split(train1)):
        print ("TRAIN:", len(train_index), "TEST:", len(test_index))
        X_train,X_test=train1.iloc[train_index], train1.iloc[test_index]
        X_train.to_csv('New_fold/fold_'+str(ix)+'_'+'x_train.csv',index=False)
        X_test.to_csv('New_fold/fold_'+str(ix)+'_'+'x_test.csv',index=False)
        ix+=1
    createFoldsData()

In [ ]:
processed_data_file_train = 'data/processed/' + 'test_data_set_fold_'+str(0)+'.pt'
processed_data_file_test = 'data/processed/'  + 'train_data_set_fold_'+str(0)+'.pt'
if ((not os.path.isfile(processed_data_file_train)) or (not os.path.isfile(processed_data_file_test))):
        print('please run create_data.py to prepare data in pytorch format!')
        createFoldsCsv()

In [ ]:
if torch.cuda.is_available():  
    device = "cuda:4"
    print("cuda:4")
else:  
    device = "cpu" 
    print(torch.cuda.is_available())

In [ ]:
# training function at each epoch
def train(model, device, train_loader, optimizer, epoch,loss_fn):
    #print('Training on {} samples...'.format(len(train_loader1.dataset)))
    model.train()
    Loss = []
    for data in train_loader:
        data = data.to(device)
        optimizer.zero_grad()
        output = model(data.x.float(), data.edge_index,data.batch)
        loss = loss_fn(output, data.y.view(-1, 1).float().to(device))
        loss.backward()
        optimizer.step()
        Loss.append(loss.item())
    nploss = np.asarray(Loss)
    avg_loss = np.average(nploss)
    return avg_loss

In [ ]:
def predicting(model, device, loader,loss_fn):
    model.eval()
    total_loss=total_example=0
    total_preds = torch.Tensor()
    total_labels = torch.Tensor()
    
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            output = model(data.x.float(), data.edge_index,data.batch)
            
            loss = loss_fn(output, data.y.view(-1, 1).float().to(device))
            total_loss+=loss
            total_example+=1
    return total_loss/total_example

In [ ]:
#LR = 0.005
LR = 9.797464088146652e-05
#LR = 0.0028894537419258915
LOG_INTERVAL = 20
NUM_EPOCHS = 3000
results = []

In [ ]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()
        
    def forward(self,yhat,y):
        return torch.sqrt(self.mse(yhat,y))

In [ ]:
loss_fn = nn.L1Loss()

In [ ]:
best_ret = []
best_mse = 0.80
best_ci = 0
best_epoch = -1

In [ ]:
folds = 5
results = []
best_mae_arr = []

for fold in tqdm(range(folds)):
    val_losses = []
    train_losses = []
    mae_arr = []
    patience = 16
    trigger_times = 0
    the_last_loss = 100
    
    model_file_name = 'saved_models/model_' +  str(fold) +  '.model'
    result_file_name = 'result_' + str(fold) +  '.csv'
    
    train_data = Molecule_data(root='data', dataset='train_data_set_fold_'+str(fold),y=None,smile_graph=None,smiles=None)
    test_data = Molecule_data(root='data', dataset='test_data_set_fold_'+str(fold),y=None,smile_graph=None,smiles=None)
    
    TRAIN_BATCH_SIZE = 17
    
    train_loder   = DataLoader(train_data,batch_size=TRAIN_BATCH_SIZE,shuffle=True)
    test_loder  = DataLoader(test_data,batch_size=TRAIN_BATCH_SIZE,shuffle=True)
    model = AttentiveFP(in_channels=114, hidden_channels=292, out_channels=1,
                    num_layers=3, num_timesteps=2,
                    dropout=0.047352327938708194).to(device)
    best_ret = []
    
    model = model.cuda(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=LR)
    best_mae = 0.00
    
    for epoch in range(NUM_EPOCHS):
        train_loss=train(model, device, train_loder, optimizer, epoch+1,loss_fn)
        test_loss = predicting(model, device, test_loder,loss_fn)
        
        print('Epoch% d: Train mae: %2.5f\t val mae: %2.5f\t'
          %(epoch, train_loss,test_loss.item()))
        
        ret = [epoch,train_loss,test_loss.item()]
        
        train_losses.append(train_loss)
        val_losses.append(test_loss.item())
        
        # Early stopping
        the_current_loss = test_loss.item()
        
        best_ret.append(ret)
        
        if the_current_loss > the_last_loss:
            trigger_times += 1
            print('trigger times:', trigger_times)
            
            if trigger_times >= patience:
                print('Early stopping!\nStart to test process.')
                break
        else:
            ret = [epoch,train_loss,test_loss.item()]
            trigger_times = 0
            best_mae = the_current_loss
            the_last_loss = the_current_loss
            
            torch.save(model.state_dict(), model_file_name)

    results.append(best_ret)
    best_mae_arr.append(best_mae)

In [ ]:
format_float = "{:.2f}".format(best_mae)

In [ ]:
type(format_float)

In [ ]:
resSt = results[0][1]

In [ ]:
res_val = resSt[2]

In [ ]:
res_val

In [ ]:
folds = 5
for fold in range(folds):
    train_loss_arr = []
    test_loss_arr = []
    for res in results[fold]:
        train_loss_arr.append(res[1])
        test_loss_arr.append(res[2])
        
    print(len(train_loss_arr))
    print(len(test_loss_arr))
    ax = plt.subplot(1,1,1)
    
    ax.WindowState = 'maximized';

    format_mae = "{:.2f}".format(best_mae_arr[fold])
    
  #  ax.plot([e for e in range(1,len(train_loss_arr) + 1)], train_loss_arr, label="train_loss")
    ax.plot([e for e in range(1,len(test_loss_arr) + 1)],
            test_loss_arr, label="Fold " + str(fold) + " (MAE = " + format_mae + ")")
    plt.xlabel("epoch")
    ax.title.set_text('5-Fold Validation')
    ax.legend()
    ax.figure.savefig('Visualization/'+str(fold)+'1.png')

In [ ]:
train_loss_arr